# Преобработку данных украл у AndreySusarev

## Загрузка библиотек, подключение Гугл-Диска

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier

In [ ]:
# Подключаем Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Загузка сохраненных данных с диска и подготовка датасета для НС

In [ ]:
df_fl = pd.read_csv('/content/drive/MyDrive/Стажировка/Рак_легких/Number_data.csv', index_col=None)
df_fl.head()

,Unnamed: 0,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Поставьте галочку, если уверены на 100%",Если 75%,Если 50%,"Альтернатива, если 50% - обязательно",Комментарий,Решение
0,0,1.0,1.0,1.0,1.0,1.0,True,False,False,1.0,1.0,1
1,1,2.0,1.0,1.0,1.0,1.0,True,False,False,1.0,1.0,1
2,2,3.0,1.0,1.0,1.0,2.0,False,True,False,2.0,2.0,2
3,3,1.0,2.0,1.0,1.0,1.0,False,True,False,1.0,3.0,2
4,4,2.0,2.0,1.0,1.0,1.0,False,True,False,1.0,3.0,2


In [ ]:
df_fl = df_fl.drop(columns = 'Unnamed: 0')
df_fl.nunique()

Срок от окончания ХЛТ                                        3
Молекулярный статус (только для неплоскоклеточного рака)     6
PD-L1 статус                                                 3
Предпочтение пациента по ответу на терапию                   2
Ответ эксперта (Лактионов)                                   4
Поставьте галочку, если уверены на 100%                      2
Если 75%                                                     2
Если 50%                                                     2
Альтернатива, если 50% - обязательно                         5
Комментарий                                                 11
Решение                                                      3
dtype: int64

In [ ]:
# Соберем таблицу для обучения сети
df_fin = df_fl.drop(columns = ['Поставьте галочку, если уверены на 100%', 'Если 75%', 'Если 50%', 'Альтернатива, если 50% - обязательно'])

In [ ]:
# Нормализуем данные в таблице (кроме ячеек 'Решение')
for i in df_fin.columns.values.tolist():
   if i != 'Решение':
        df_fin[i] = df_fin[i]/df_fin[i].max()

In [ ]:
df_fin.head(10)

,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),Комментарий,Решение
0,0.333333,0.166667,0.333333,0.5,0.25,0.090909,1
1,0.666667,0.166667,0.333333,0.5,0.25,0.090909,1
2,1.000000,0.166667,0.333333,0.5,0.50,0.181818,2
3,0.333333,0.333333,0.333333,0.5,0.25,0.272727,2
4,0.666667,0.333333,0.333333,0.5,0.25,0.272727,2
5,1.000000,0.333333,0.333333,0.5,0.50,0.181818,2
6,0.333333,0.500000,0.333333,0.5,0.50,0.363636,1
7,0.666667,0.500000,0.333333,0.5,0.50,0.363636,1
8,1.000000,0.500000,0.333333,0.5,0.50,0.090909,1
9,0.333333,0.666667,0.333333,0.5,0.75,0.363636,2


In [ ]:
# Сформируем массив данных и их значений
x=df_fin.drop(columns=['Решение','Ответ эксперта (Лактионов)','Комментарий']).reset_index(drop=True)
y=df_fin['Решение']


In [ ]:
x = x.to_numpy()
y = y.to_numpy()

In [ ]:
x

array([[0.33333333, 0.16666667, 0.33333333, 0.5       ],
       [0.66666667, 0.16666667, 0.33333333, 0.5       ],
       [1.        , 0.16666667, 0.33333333, 0.5       ],
       ...,
       [0.33333333, 1.        , 1.        , 1.        ],
       [0.66666667, 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        ]])

In [ ]:
y

array([1, 1, 2, ..., 3, 3, 1])

In [ ]:
# Разобъем массив на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True)

In [ ]:
# Вывод размеров обучающей и проверочной выборок
print(f'Структура и размер массива обучающей выборки: {x_train.shape}')
print(f'Структура и размер массива проверочной выборки: {x_test.shape}')

Структура и размер массива обучающей выборки: (199065, 4)
Структура и размер массива проверочной выборки: (49767, 4)


## Предобученные алгоритмы ML


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgbm_tuned = LGBMClassifier(boosting_type = 'gbdt'
                            ,force_col_wise=True) # LightGBM Classifier with optimum paramteres
lgbm_tuned.fit(x_train, y_train)

[LightGBM] [Info] Total Bins 18
[LightGBM] [Info] Number of data points in the train set: 199065, number of used features: 4
[LightGBM] [Info] Start training from score -1.016882
[LightGBM] [Info] Start training from score -1.018661
[LightGBM] [Info] Start training from score -1.283013
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(force_col_wise=True)

In [ ]:
expected_y  = y_test
predicted_y = model.predict(x_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(expected_y, predicted_y))
print(metrics.confusion_matrix(expected_y, predicted_y))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     17850
           2       1.00      1.00      1.00     17978
           3       1.00      1.00      1.00     13939

    accuracy                           1.00     49767
   macro avg       1.00      1.00      1.00     49767
weighted avg       1.00      1.00      1.00     49767

[[17850     0     0]
 [    0 17978     0]
 [    0     0 13939]]
